In [4]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss

In [15]:
doc1 = Document(
    page_content ="Tamim Iqbal is a Bangladeshi cricketer who plays as a left-handed opening batsman. he is a batter. He is the captain of the Bangladesh national team in Test and T20I formats.",
    metadata={"team": "Bangladesh", "format": "Test"}
)

doc2 = Document(
    page_content ="Shakib Al Hasan is a Bangladeshi cricketer, widely regarded as one of the best all-rounders in the world. he is a bowler also a batter. He has been a key player for Bangladesh in all formats of the game.",
    metadata={"team": "Bangladesh", "format": "ODI"}
)

doc3 = Document(
    page_content ="Virat Kohli is an Indian cricketer and former captain of the Indian national team. He is known for his aggressive batting style and has numerous records in international cricket.",
    metadata={"team": "India", "format": "T20I"}
)

doc4 = Document(
    page_content ="Rohit Sharma is an Indian cricketer who plays as a right-handed opening batsman. He is the captain of the Indian national team in ODI and T20I formats.",
    metadata={"team": "India", "format": "ODI"}
)

In [5]:
embedding= OllamaEmbeddings(model="llama3.2")
index = faiss.IndexFlatL2(len(embedding.embed_query("Hello World!")))

vector_store = FAISS(
    embedding_function= embedding,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)



In [16]:
from uuid import uuid4
docs = [doc1, doc2, doc3, doc4]
uuids = [str(uuid4()) for _ in range(len(docs))] # Generate unique IDs for each document

vector_store.add_documents(documents=docs, ids=uuids)

['6a36c5e2-0de7-42f6-97b9-fafa1e59f66a',
 'e82dceaf-339e-421f-9779-4eb87f4133bf',
 'be5c4863-5eba-4e0c-8f36-b1b83d47f198',
 '6183bc2b-4b58-4e1b-86b9-fc93f7b420e1']

In [18]:
vector_store.similarity_search(
    query="who is the bowler here ?" ,
    k=2,
)

[Document(id='6a36c5e2-0de7-42f6-97b9-fafa1e59f66a', metadata={'team': 'Bangladesh', 'format': 'Test'}, page_content='Tamim Iqbal is a Bangladeshi cricketer who plays as a left-handed opening batsman. he is a batter. He is the captain of the Bangladesh national team in Test and T20I formats.'),
 Document(id='e82dceaf-339e-421f-9779-4eb87f4133bf', metadata={'team': 'Bangladesh', 'format': 'ODI'}, page_content='Shakib Al Hasan is a Bangladeshi cricketer, widely regarded as one of the best all-rounders in the world. he is a bowler also a batter. He has been a key player for Bangladesh in all formats of the game.')]

In [23]:
result= vector_store.similarity_search_with_score(
    query="who is the batter here ?",
    k=2,
)
for doc, score in result:
    print(f"Document: {doc.page_content}, \nScore: {score}")

Document: Tamim Iqbal is a Bangladeshi cricketer who plays as a left-handed opening batsman. he is a batter. He is the captain of the Bangladesh national team in Test and T20I formats., 
Score: 1.0575346946716309
Document: Shakib Al Hasan is a Bangladeshi cricketer, widely regarded as one of the best all-rounders in the world. he is a bowler also a batter. He has been a key player for Bangladesh in all formats of the game., 
Score: 1.0988166332244873


In [ ]:
vector_store.save_local("faiss_index")

new_vector_store = FAISS.load_local(
    "faiss_index", embedding, allow_dangerous_deserialization=True
)

In [33]:
for id, doc in vector_store.docstore._dict.items():
    print(doc.page_content)
    print("---------------")
print("Total documents in vector store:", len(vector_store.docstore._dict))

Tamim Iqbal is a Bangladeshi cricketer who plays as a left-handed opening batsman. He is the captain of the Bangladesh national team in Test and T20I formats.
---------------
Shakib Al Hasan is a Bangladeshi cricketer, widely regarded as one of the best all-rounders in the world. He has been a key player for Bangladesh in all formats of the game.
---------------
Virat Kohli is an Indian cricketer and former captain of the Indian national team. He is known for his aggressive batting style and has numerous records in international cricket.
---------------
Rohit Sharma is an Indian cricketer who plays as a right-handed opening batsman. He is the captain of the Indian national team in ODI and T20I formats.
---------------
Tamim Iqbal is a Bangladeshi cricketer who plays as a left-handed opening batsman. he is a batter. He is the captain of the Bangladesh national team in Test and T20I formats.
---------------
Shakib Al Hasan is a Bangladeshi cricketer, widely regarded as one of the best al

In [36]:
doc5 = Document(
    page_content="This document is ment to be deleted.",
    metadata={"format": "Test"}
)

vector_store.add_documents(documents=[doc5], ids=[str(uuid4())])

['4a61323e-a9fd-4753-a7ef-792b5dad1c53']

In [37]:
vector_store.delete(ids=['4a61323e-a9fd-4753-a7ef-792b5dad1c53'])

True

In [39]:
for id , doc in vector_store.docstore._dict.items():
    print(f"id :{id}")
    print(f"page content: {doc.page_content}")
    print("---------------")

id :6290dc5f-2579-4500-8dbd-b52f2e448be4
page content: Tamim Iqbal is a Bangladeshi cricketer who plays as a left-handed opening batsman. He is the captain of the Bangladesh national team in Test and T20I formats.
---------------
id :b116a24b-418f-438f-b714-e749c8e70d3a
page content: Shakib Al Hasan is a Bangladeshi cricketer, widely regarded as one of the best all-rounders in the world. He has been a key player for Bangladesh in all formats of the game.
---------------
id :dcf54611-11e7-4039-aab6-830dde505380
page content: Virat Kohli is an Indian cricketer and former captain of the Indian national team. He is known for his aggressive batting style and has numerous records in international cricket.
---------------
id :0feaab11-8426-43d0-bb18-0df6f6fed029
page content: Rohit Sharma is an Indian cricketer who plays as a right-handed opening batsman. He is the captain of the Indian national team in ODI and T20I formats.
---------------
id :6a36c5e2-0de7-42f6-97b9-fafa1e59f66a
page conten

In [ ]:
docstore = vector_store.docstore._dict

In [ ]:
docstore

{'6290dc5f-2579-4500-8dbd-b52f2e448be4': Document(id='6290dc5f-2579-4500-8dbd-b52f2e448be4', metadata={'team': 'Bangladesh', 'format': 'Test'}, page_content='Tamim Iqbal is a Bangladeshi cricketer who plays as a left-handed opening batsman. He is the captain of the Bangladesh national team in Test and T20I formats.'),
 'b116a24b-418f-438f-b714-e749c8e70d3a': Document(id='b116a24b-418f-438f-b714-e749c8e70d3a', metadata={'team': 'Bangladesh', 'format': 'ODI'}, page_content='Shakib Al Hasan is a Bangladeshi cricketer, widely regarded as one of the best all-rounders in the world. He has been a key player for Bangladesh in all formats of the game.'),
 'dcf54611-11e7-4039-aab6-830dde505380': Document(id='dcf54611-11e7-4039-aab6-830dde505380', metadata={'team': 'India', 'format': 'T20I'}, page_content='Virat Kohli is an Indian cricketer and former captain of the Indian national team. He is known for his aggressive batting style and has numerous records in international cricket.'),
 '0feaab11-

In [ ]:
# Collect the first 4 IDs to delete
ids_to_delete = []
count = 0
for id, doc in vector_store.docstore._dict.items():
    if count < 4:
        ids_to_delete.append(id)
        count += 1

# Delete them outside the iteration
for id in ids_to_delete:
    vector_store.delete(ids=[id])

vector_store.docstore._dict

{'e82dceaf-339e-421f-9779-4eb87f4133bf': Document(id='e82dceaf-339e-421f-9779-4eb87f4133bf', metadata={'team': 'Bangladesh', 'format': 'ODI'}, page_content='Shakib Al Hasan is a Bangladeshi cricketer, widely regarded as one of the best all-rounders in the world. he is a bowler also a batter. He has been a key player for Bangladesh in all formats of the game.'),
 'be5c4863-5eba-4e0c-8f36-b1b83d47f198': Document(id='be5c4863-5eba-4e0c-8f36-b1b83d47f198', metadata={'team': 'India', 'format': 'T20I'}, page_content='Virat Kohli is an Indian cricketer and former captain of the Indian national team. He is known for his aggressive batting style and has numerous records in international cricket.'),
 '6183bc2b-4b58-4e1b-86b9-fc93f7b420e1': Document(id='6183bc2b-4b58-4e1b-86b9-fc93f7b420e1', metadata={'team': 'India', 'format': 'ODI'}, page_content='Rohit Sharma is an Indian cricketer who plays as a right-handed opening batsman. He is the captain of the Indian national team in ODI and T20I format